In [1]:
%pip install river pandas==2.2.2 scipy==1.13.1 numpy==1.26.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
from sklearn.model_selection import train_test_split
from river import metrics, preprocessing
from classes.superclasse.ModeloBase import ModeloBase

# 1.Baixar Dados

A função `baixar_dados` baixa os dados históricos de uma ação do Yahoo Finance, permitindo ajustar o período e o intervalo dos dados.

In [3]:
# Função para baixar dados do Yahoo Finance
def baixar_dados(symbol, periodo="5y", intervalo="1d"):
    data = yf.download(symbol, period=periodo, interval=intervalo)
    return data["Close"].values

## 2. Criar Janelas Temporais e Normalizar Dados
A função `criar_janela_temporal` cria as janelas de entrada e saída para o modelo de previsão. As entradas são geradas com base no número de `lags` definido e as saídas são os valores de fechamento seguintes. A função `Normalizar` transforma os dados para o intervalo de 0 e 1.

In [4]:
# Função para criar padrões de entrada e saída
def criar_janela_temporal(y, lags):
    X, Y = [], []
    for i in range(len(y) - lags):
        X.append(y[i:i+lags])
        Y.append(y[i+lags])
    return np.array(X).reshape(-1, lags), np.array(Y)

In [5]:
def normalizar_serie(serie_temporal: np.ndarray) -> np.ndarray:
    # Criando o scaler
    scaler = preprocessing.StandardScaler()

    # Aprendendo a escala com os dados
    for x in serie_temporal:
        scaler.learn_one({"valor": x[0]})  # Não sobrescrevemos o scaler

    # Transformando a série
    serie_normalizada = np.array([scaler.transform_one({"valor": x[0]})["valor"] for x in serie_temporal])

    # Garantindo que o shape permaneça (1244, 1)
    return serie_normalizada.reshape(-1, 1)

# 3.Inicializar Modelos

In [6]:
def inicializar_modelos(modelo_classe, detector_classe, **kwargs):

     # Instancia o modelo com os parâmetros fornecidos
     modelo_instancia = modelo_classe(**kwargs)

     # Instancia o detector com os parâmetros fornecidos
     detector_instancia = detector_classe(**kwargs)

     return modelo_instancia, detector_instancia

## 4. Treinamento do Modelo de Machine Learning
A função `treinamento_modelo` realiza o treinamento inicial do modelo com os primeiros dados disponíveis, usando o tamanho do batch especificado.

In [7]:
def treinamento_modelo_batch(modelo, X, y):
    # Treinamento do modelo usando o método 'treinar' da subclasse
    modelo.treinar(X, y)

    # Cálculo do erro médio (adapte para modelos online, se necessário)

    if isinstance(modelo, ModeloBase):  # Verifica se é uma instância da superclasse dos modelos offline
      erro_medio = np.abs(np.mean(y - modelo.prever(X))) # Calcula erro para modelos offline

    else: # Senão calcula para modelos online
      predicoes = []
      for i in range(len(X)):
          predicoes.append(modelo.prever(X[i].reshape(1, -1))[0]) # Faz as predições para cada exemplo em X
      erro_medio = np.abs(np.mean(y.ravel() - np.array(predicoes))) # Calcula o erro médio

    return erro_medio

In [8]:
def treinamento_online_many(modelo, X, y, tamanho_batch):  # Adiciona 'tamanho_batch' como parâmetro
    # Treina o modelo com os primeiros 'tamanho_batch' exemplos
    modelo.treinar(X[:tamanho_batch], y[:tamanho_batch])
    return modelo

In [9]:
def prequential_batch(X, Y, tamanho_batch, modelo_classe, detector_classe, **kwargs):
    """
    Realiza a previsão de valores continuamente, detectando mudanças nos dados (drift)
    e retreinando o modelo quando necessário.

    Args:
        X: Dados de entrada.
        Y: Dados de saída.
        tamanho_batch: Tamanho do batch para treinamento inicial e retreinamento.
        modelo_classe: Classe do modelo a ser usado (subclasse de ModeloBase).
        detector_classe: Classe do detector de drift a ser usado (subclasse de DetectorDriftBase).
        **kwargs: Parâmetros adicionais para o modelo e detector.

    Returns:
        predicoes: Lista de previsões.
        deteccoes: Lista de índices onde o drift foi detectado.
    """
    predicoes, erros, deteccoes = [], [], []
    mae = metrics.MAE()

    # Inicializando o modelo e o detector usando as classes
    modelo, detector = inicializar_modelos(modelo_classe, detector_classe, **kwargs)

    # Treina o modelo e atualiza o detector
    erro_inicial = treinamento_modelo_batch(modelo, X[:tamanho_batch], Y[:tamanho_batch])
    detector.atualizar(erro_inicial)  # Usa o método 'atualizar' da subclasse

    drift_ativo = False

    for i in range(tamanho_batch, len(X)):
        # Realiza a predição usando o método 'prever' da subclasse
        entrada = X[i].reshape(1, -1)
        y_pred = modelo.prever(entrada)[0]
        erro = abs(Y[i][0] - y_pred)

        predicoes.append(y_pred)
        erros.append(erro)
        mae.update(Y[i][0], y_pred)

        # Atualiza o detector usando o método 'atualizar' da subclasse
        detector.atualizar(erro)

        # Se drift for detectado pela primeira vez
        if detector.drift_detectado and not drift_ativo:  # Usa a propriedade 'drift_detectado'
            deteccoes.append(i)
            print(f"\nMudança detectada no índice {i}, começando a coletar dados para retreino...")
            drift_ativo = True
            janela_X, janela_y = [], []

        # Se drift já foi detectado, inicia-se a coleta dos dados
        if drift_ativo:
            janela_X.append(X[i])
            janela_y.append(Y[i])

            if len(janela_X) >= tamanho_batch:
                print(f"Janela completa com {len(janela_X)} amostras. Retreinado com dados do índice {i - tamanho_batch} até {i}.")
                drift_ativo = False

                # Inicializando o modelo e o detector com novas instâncias
                modelo, detector = inicializar_modelos(modelo_classe, detector_classe, **kwargs)

                # Treina o modelo e atualiza o detector
                erro_inicial = treinamento_modelo_batch(modelo, np.array(janela_X), np.array(janela_y))
                detector.atualizar(erro_inicial)  # Usa o método 'atualizar' da subclasse


    print("Modelo utilizado:", modelo)
    print("Detector utilizado:", detector)
    print(f"MAE Modelo Batch: {mae.get()[0]}")
    return predicoes, deteccoes

In [10]:
def prequential_passivo(X, Y, tamanho_batch, modelo_classe, **kwargs):
    """
    Realiza a previsão de valores continuamente para algoritmos online,
    sem detecção de drift e retreinamento.

    Args:
        X: Dados de entrada.
        Y: Dados de saída.
        tamanho_batch: Tamanho do batch para treinamento inicial.
        modelo_classe: Classe do modelo a ser usado (subclasse de ModeloBase).
        **kwargs: Parâmetros adicionais para o modelo.

    Returns:
        predicoes: Lista de previsões.
    """
    predicoes = []
    mae = metrics.MAE()

    # Inicializa o modelo usando a classe e kwargs
    modelo = modelo_classe(**kwargs)

    # Treina o modelo com os primeiros exemplos usando treinamento_online_many
    modelo = treinamento_online_many(modelo, X, Y, tamanho_batch)

    for i in range(tamanho_batch, len(X)):
        # Converte a entrada para o formato que o modelo online espera
        entrada_dict = {f"t{j+1}": value for j, value in enumerate(X[i])}

        # Realiza a predição usando o método 'prever' da subclasse
        y_pred = modelo.prever([X[i]])  # Passa a entrada como uma lista de uma única amostra

        predicoes.append(y_pred)
        mae.update(Y[i][0], y_pred)

        # Atualiza o modelo online usando o método 'treinar' da subclasse
        modelo.treinar([X[i]], [Y[i]])

    print("Modelo utilizado:", modelo)
    print(f"MAE Modelo Online: {mae.get()}")
    return predicoes

# Plot

In [11]:
def plotar_resultados(Y, lista_predicoes, labels_algoritmos, deteccoes, tamanho_batch):
    plt.figure(figsize=(15, 8))
    indices = range(tamanho_batch, tamanho_batch + len(Y[tamanho_batch:]))

    # Plotar valores verdadeiros
    plt.plot(indices, Y[tamanho_batch:tamanho_batch + len(indices)],
             label="Verdadeiro", linewidth=1.2)

    # Plotar cada conjunto de previsões
    for i, predicoes in enumerate(lista_predicoes):
        Y_plot = Y[tamanho_batch:tamanho_batch + len(predicoes)]
        predicoes = predicoes[:len(Y_plot)]  # Garantir mesmo tamanho
        label = labels_algoritmos[i] if i < len(labels_algoritmos) else f"Previsões {i+1}"
        plt.plot(indices[:len(predicoes)], predicoes, label=label, linewidth=1.2)

    # Aumentar o tamanho dos pontos de detecção
    if deteccoes:
        plt.scatter(deteccoes, [Y[i] for i in deteccoes], color='red',
                   label="Drift Detectado", zorder=3, s=80)

        # Destacar áreas pós-retreino com fundo colorido
        for idx, d in enumerate(deteccoes):
            next_end = min(d + tamanho_batch, indices[-1])
            plt.axvspan(d, next_end, alpha=0.1, color='green', label='_nolegend_')

        print("\nDrift detectado nos índices:", deteccoes)
    else:
        print("\nNenhum drift detectado.")

    plt.legend(fontsize=12)
    plt.title("Predições e Detecção de Drift com Retreino", fontsize=14)
    plt.grid(True, alpha=0.4)

    # Adicionar anotações para mostrar diferenças
    for i in range(len(deteccoes)):
        d = deteccoes[i]
        if d + 5 < len(indices):
            plt.annotate(f"Retreino",
                         xy=(d, Y[d]),
                         xytext=(d+10, Y[d]+0.1),
                         arrowprops=dict(facecolor='black', shrink=0.05, width=1.5),
                         fontsize=10)

    plt.show()

# Roda Roda Jequiti

In [12]:
series = [
    "AAPL",
    "B3SA3.SA"
    "^IXIC",
    "^DJI",
    "^GSPC",
    "^BVSP",
    "USDBRL=X"
]

In [13]:
from classes.detectores.ADWINDetector import ADWINDetector
from classes.detectores.DDMDetector import DDMDetector
from classes.detectores.EDDMDetector import EDDMDetector 
from classes.detectores.FHDDMDetector import FHDDMDetector
from classes.detectores.HDDM_ADetector import HDDM_ADetector 
from classes.detectores.PageHinkleyDetector import PageHinkleyDetector
from classes.detectores.HDDM_WDetector import HDDM_WDetector
from classes.modelosOffline.LinearRegressionModelo import LinearRegressionModelo
from classes.modelosOnline.BayesianLinearRegressionModelo import BayesianLinearRegressionModelo 
from classes.modelosOnline.HoeffdingTreeRegressorModelo import HoeffdingTreeRegressorModelo 
from classes.modelosOnline.KNNRegressorOnlineModelo import KNNRegressorOnlineModelo 
from classes.modelosOnline.LinearRegressionOnlineModelo import LinearRegressionOnlineModelo

In [14]:
tamanho_batch, lags = 50, 5

# Baixando os dados da ação
serie_temporal = baixar_dados(series[-2])

# normalizando a série temporal
serie_temporal = normalizar_serie(serie_temporal)

# Gerando janelas temporais
X, Y = criar_janela_temporal(serie_temporal, lags)

print(LinearRegressionModelo)

# Prequential com detecção de drift (usando classes)
predicoes_ativa1, deteccoes = prequential_batch(X, Y, tamanho_batch, modelo_classe=LinearRegressionModelo, detector_classe=DDMDetector)
predicoes_ativa2, deteccoes = prequential_batch(X, Y, tamanho_batch, modelo_classe=LinearRegressionModelo, detector_classe=ADWINDetector)
predicoes_ativa4, deteccoes = prequential_batch(X, Y, tamanho_batch, modelo_classe=LinearRegressionModelo, detector_classe=PageHinkleyDetector)
predicoes_ativa5, deteccoes = prequential_batch(X, Y, tamanho_batch, modelo_classe=LinearRegressionModelo, detector_classe=EDDMDetector)
predicoes_ativa6, deteccoes = prequential_batch(X, Y, tamanho_batch, modelo_classe=LinearRegressionModelo, detector_classe=FHDDMDetector)
predicoes_ativa7, deteccoes = prequential_batch(X, Y, tamanho_batch, modelo_classe=LinearRegressionModelo, detector_classe=HDDM_ADetector)
predicoes_ativa8, deteccoes = prequential_batch(X, Y, tamanho_batch, modelo_classe=LinearRegressionModelo, detector_classe=HDDM_WDetector)

# Prequential passivo (usando classes)
#predicoes_passiva1 = prequential_passivo(X, Y, tamanho_batch, modelo_classe=BayesianLinearRegressionModelo)
predicoes_passiva2 = prequential_passivo(X, Y, tamanho_batch, modelo_classe=KNNRegressorOnlineModelo)
predicoes_passiva3 = prequential_passivo(X, Y, tamanho_batch, modelo_classe=LinearRegressionOnlineModelo)
predicoes_passiva4 = prequential_passivo(X, Y, tamanho_batch, modelo_classe=HoeffdingTreeRegressorModelo)

# Plotando os resultados
plotar_resultados(Y, [
                      predicoes_ativa1,
                      predicoes_ativa2,
                      #predicoes_ativa3,
                      predicoes_ativa4,
                      predicoes_ativa5,
                      predicoes_ativa6,
                      predicoes_ativa7,
                      predicoes_ativa8,
                      predicoes_passiva1,
                      predicoes_passiva2,
                      predicoes_passiva3,
                      predicoes_passiva4,
                      #predicoes_passiva5
                      ],
                  [
                    "DDM",
                    "ADWIN",
                    "KSWIN",
                    "PageHinkley",
                    "EDDM",
                    "FHDDM",
                    "HDDM_A",
                    "HDDM_W",
                    "BayesianLinearRegression",
                    "KNNRegressor",
                    "LinearRegression",
                    "HoeffdingTreeRegressor",
                    #"MLPRegressor"
                   ],
                  deteccoes, tamanho_batch)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


<class 'classes.modelosOffline.LinearRegressionModelo.LinearRegressionModelo'>
Modelo utilizado: <classes.modelosOffline.LinearRegressionModelo.LinearRegressionModelo object at 0x30108f6b0>
Detector utilizado: <classes.detectores.DDMDetector.DDMDetector object at 0x17fb59850>
MAE Modelo Batch: 0.08620558898706357
Modelo utilizado: <classes.modelosOffline.LinearRegressionModelo.LinearRegressionModelo object at 0x300089c40>
Detector utilizado: <classes.detectores.ADWINDetector.ADWINDetector object at 0x16598fe00>
MAE Modelo Batch: 0.08620558898706357
Modelo utilizado: <classes.modelosOffline.LinearRegressionModelo.LinearRegressionModelo object at 0x300089c40>
Detector utilizado: <classes.detectores.PageHinkleyDetector.PageHinkleyDetector object at 0x17fb59850>
MAE Modelo Batch: 0.08620558898706357
Modelo utilizado: <classes.modelosOffline.LinearRegressionModelo.LinearRegressionModelo object at 0x30108f6b0>
Detector utilizado: <classes.detectores.EDDMDetector.EDDMDetector object at 0x17f6

AttributeError: 'LinearRegression' object has no attribute 'learn_one'